# 🦜🔗 LangChain Fundamentals: Interactive Demo

Welcome to this hands-on demonstration!  

This notebook provides a comprehensive, interactive demonstration of the **core components that make LangChain powerful for building production-ready AI applications**.  

You'll explore how to communicate with AI models, enable them to use tools, and make them work with your own data through practical, real-world examples.

## 📚 What This Demo Covers

### **1. Prompt Templates**
Learn how to communicate with AI models efficiently using reusable and structured prompts that ensure consistent behavior.

### **2. Chat Models**
These are the *brains* of your application—Large Language Models (LLMs) that generate responses, reason through problems, and follow complex instructions.

### **3. LCEL (LangChain Expression Language)**
Connect multiple components together to build sophisticated workflows and data processing pipelines.  

### **4. Structured Output**
Receive clean, typed outputs (like JSON or Pydantic models) for seamless integration with your application code.

### **5. Tool Calling**
Enable AI models to use external tools—calculators, APIs, databases, or any custom function you provide.

### **6. RAG (Retrieval Augmented Generation)**
Provide external knowledge sources (PDFs, documents, databases, web pages) so the AI can *answer questions based on your specific data*.

---

**Each section includes:**
- Clear explanations of the concept
- Real-world analogies to make complex ideas intuitive
- Working code examples you can run and modify
- Technical context to understand what's happening under the hood

## Setup: API Keys

**For Local/Jupyter:** Set your OpenAI key in a `.env` file (skip if already in your shell env).

**For Google Colab:** Use Colab's secrets manager:
1. Click the 🔑 key icon in the left sidebar
2. Add a new secret with name `OPENAI_API_KEY` and your API key as the value
3. The code below will automatically detect and use it!

In [1]:
from dotenv import load_dotenv
load_dotenv(dotenv_path=".env")

True

In [3]:
# Diagnostic cell: prints kernel info and tests imports
import sys, traceback, os
print("Notebook kernel executable:", sys.executable)
#print("Python version:", sys.version.replace('
#', ' '))
print("Working directory:", os.getcwd())
print("sys.path (truncated):", sys.path[:10])

# Test top-level package import
try:
    import langchain_core
    print("langchain_core installed at:", getattr(langchain_core, '__file__', str(langchain_core)))
except Exception:
    print("Failed to import langchain_core:")
    traceback.print_exc()

# Test specific symbol import
try:
    from langchain_core.prompts import ChatPromptTemplate
    print("ChatPromptTemplate import: OK")
except Exception:
    print("Failed to import ChatPromptTemplate from langchain_core.prompts:")
    traceback.print_exc()

# Also verify local helper module imports that the notebook uses
try:
    import langchain_prompts
    print("langchain_prompts loaded from:", getattr(langchain_prompts, '__file__', str(langchain_prompts)))
except Exception:
    print("Failed to import local module langchain_prompts:")
    traceback.print_exc()


Notebook kernel executable: /opt/anaconda3/bin/python
Working directory: /Users/massadraza/Desktop/Langchain Prep/langchain-langgraph-demo
sys.path (truncated): ['/opt/anaconda3/lib/python313.zip', '/opt/anaconda3/lib/python3.13', '/opt/anaconda3/lib/python3.13/lib-dynload', '', '/opt/anaconda3/lib/python3.13/site-packages', '/opt/anaconda3/lib/python3.13/site-packages/aeosa']
langchain_core installed at: /opt/anaconda3/lib/python3.13/site-packages/langchain_core/__init__.py
ChatPromptTemplate import: OK
langchain_prompts loaded from: /Users/massadraza/Desktop/Langchain Prep/langchain-langgraph-demo/langchain_prompts.py


## 1) Prompt Templates
Think of sending professional emails. You probably have templates like:
*"Dear **{Name}**, I'm writing to inform you about **{Topic}**. Please let me know by **{Deadline}**."*

You don't compose a brand new email from scratch every time—you use the same structure and just fill in the blanks with different names, topics, and dates. Prompt Templates work the same manner—they're reusable blueprints where you swap in dynamic variables without rewriting the entire prompt.

###  Code Context
In the code below:
* `ChatPromptTemplate`: This is our blueprint.
* `{user_query}` and `{todays_date}`: These are the **variables** (the blanks we will fill in later).
* We use a System Prompt to tell the AI *who* it is (a Date Assistant) before it answers.


In [6]:
from langchain_prompts import DATE_ASSISTANT_SYSTEM_PROMPT
from datetime import datetime
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", DATE_ASSISTANT_SYSTEM_PROMPT),
    ("human", "{user_query}")
])

example_user_query = """
What is tomorrow's date?
"""

messages = prompt.format_messages(user_query = example_user_query, 
                                    todays_date = datetime.now().strftime("%Y-%m-%d"))  # Shows message objects ready for a chat model

messages


# Other Message Objects
    # System messages
    # Human messages
    # AIMessage
    # ToolMessage



[SystemMessage(content="\n\nYou are a helpful and accurate date assistant. \nYou are given a user's request, and today's date. Your job is to accurately infer the date in YYYY-MM-DD format from the user's request.\n\ntoday's date : 2026-01-07\n\nchat_history:\n\n", additional_kwargs={}, response_metadata={}),
 HumanMessage(content="\nWhat is tomorrow's date?\n", additional_kwargs={}, response_metadata={})]

## 2) Chat Models (OpenAI)

Think of Chat Models as different brands of TVs (Sony, Samsung, LG). Each one works differently on the inside, but LangChain gives you a **"Universal Remote"** to control them all.
This "Universal Remote" is called the **Runnable Interface**. It means you can press the "Play" button (`invoke`) on *any* model, and it will work exactly the same way. You don't need to learn a new remote control just because you bought a new TV.

### 🧠 Interface Concepts
all Chat Models implement the **Runnable Interface**. This guarantees they all speak the same language:
1.  **Standard Inputs:** They all accept a list of `Messages` (System, Human, AI).
2.  **Standard Outputs:** They all return an `AIMessage`.
3.  **Standard Methods:**
    * `.invoke()`: Send a message and wait for the full answer.
    * `.stream()`: Get the answer word-by-word (like a typewriter).
    * `.batch()`: Send 50 questions at once and get 50 answers back.

###  Code Context
In the code below:
* `init_chat_model`: A universal function to load a model.
* `model="gpt-4o-mini"`: We are selecting a specific, fast model from OpenAI.
* `temperature=0`: We set this to 0 to make the AI **factual and consistent**. Higher numbers (up to 1) make it more creative and random.

- Supports over 100 model providers : https://docs.langchain.com/oss/python/integrations/chat
- https://docs.langchain.com/oss/python/langchain/models

In [9]:
try:
    from langchain_openai import ChatOpenAI
except Exception:
    try:
        from langchain.chat_models import ChatOpenAI
    except Exception as e:
        raise ImportError(
            "Could not import ChatOpenAI. Install 'langchain-openai' or a recent 'langchain' package."
        ) from e

# Shim for the missing init_chat_model API used elsewhere in the notebook
def init_chat_model(model: str = "gpt-4o-mini", temperature: float = 0, **kwargs):
    return ChatOpenAI(model=model, temperature=temperature, **kwargs)

llm = init_chat_model(model="gpt-4o-mini", temperature=0) # Tip: Try other models/model providers

## 3) LLM Invocation  

* **Invoke:** This is like sending a text message and waiting for the reply. You don't see anything until the whole message arrives.
* **Stream:** This is like watching someone type in real-time. You see the words appear one by one. This feels much faster to a user.

###  Code Context
In the code below:
* `llm.invoke(messages)`: Sends our formatted prompt to the AI and waits for the full `AIMessage` response.
* `llm.stream(messages)`: Returns a generator that prints tokens as they arrive.

📚 **[Docs](https://docs.langchain.com/oss/python/langchain/models#invocation)**

In [10]:
# Invoke : 
response = llm.invoke(messages)
response


AIMessage(content='2026-01-08', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 6, 'prompt_tokens': 73, 'total_tokens': 79, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_c4585b5b9c', 'id': 'chatcmpl-CvWwKckVmXU5UOL3JW1L8z7l2UEuV', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019b9ab8-a667-7ee3-b905-b2d554816e0e-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 73, 'output_tokens': 6, 'total_tokens': 79, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [12]:
print(response.content)

2026-01-08


## 3.1) Streaming (Real-time Output)

Imagine you hired a ghostwriter to write a story.
* **Invoke (Standard):** The writer goes into a private room, locks the door, writes the entire story, and comes out 20 minutes later to hand you the finished manuscript. You sit in silence waiting the whole time.
* **Stream (Real-time):** The writer sits right in front of you and types. You see every letter appear on the screen the moment they press the key. The story takes the same amount of time to finish, but it **feels instant** because you are engaged immediately.

### Technical Concept
LLMs don't "think" of a whole sentence at once; they generate text **one token (word part) at a time**.
* **The Bottleneck:** By default (`.invoke()`), the program buffers (holds) all these tokens in memory until the model is completely finished. This creates **Latency** (the delay between asking and seeing the answer).
* **The Solution:** The `.stream()` method bypasses this buffer. It creates a Python **Generator** that yields each token (`AIMessageChunk`) the exact millisecond it is created. This drastically lowers the "Time to First Token."

### Code Context
In the code below:
* `llm.stream(messages)`: This replaces `.invoke()`. It returns an iterable stream, not a final string.
* `print(..., end="", flush=True)`: We force Python to print *immediately* without moving to a new line, creating that smooth "typewriter" effect.

📚 **[Docs: Streaming](https://docs.langchain.com/oss/python/langchain/streaming)**


In [14]:
# Streaming : 

for chunk in llm.stream(messages):
    print(chunk.content, end="", flush=True)


Tomorrow's date is 2026-01-08.

## 4) Chaining with LCEL (LangChain Expression Language)

Imagine a factory assembly line:
* **Station A:** Puts dough on the belt (The Prompt).
* **Station B:** Bakes the dough (The Model).
* **Station C:** Packages the bread (The Output Parser).

LCEL lets us pipe (`|`) these steps together. The output of one step automatically becomes the input of the next.

###  Code Context
In the code below:
* `chain = prompt | llm | StrOutputParser()`
* The `|` symbol is the magic pipe.
* `StrOutputParser`: Converts the complex AI message object directly into a simple string so we don't have to do `print(response.content)`.

📚 **[Docs: LCEL Conceptual Guide]()**


In [8]:
from langchain_core.output_parsers import StrOutputParser

chain = prompt | llm | StrOutputParser()


print(chain.invoke({ 'user_query' : example_user_query, 
                    'todays_date' : datetime.now().strftime("%Y-%m-%d") }))


Tomorrow's date is 2026-01-03.


## 5) Structured Output

Usually, AI likes to chat and write paragraphs. But sometimes, you want a form filled out, not a conversation.
If you ask for a date range, you don't want *"Sure! The start date is..."*. You want `{"start": "2025-01-01", "end": "2025-01-05"}`.
Structured output forces the AI to stop chatting and generate data matching a specific schema.

###  Code Context
In the code below:
* `class DateRange(BaseModel)`: We define the "Form" we want the AI to fill out using Pydantic.
* `llm.with_structured_output(DateRange)`: We tell the LLM, "Your output **must** match this class."
* The result is an actual Python object, not a string.

📚 **[Docs: Structured Output](https://docs.langchain.com/oss/python/langchain/models#structured-output)**

In [15]:
# Supports
    # - pydantic
    # - Typedict
    # - Dataclass 
    # - Json schema

In [18]:
from typing import List, Optional, TypedDict
from pydantic import BaseModel, Field
from langchain_prompts import DATE_EXTRACTOR_PROMPT_TEMPLATE

# You can define structure with Pydantic
class DateRange(BaseModel):
    start_date: str = Field(default = None, description="Start Date of the date range")
    end_date: str = Field(default = None, description="End Date of the date range")

# OR you can define structure with TypedDict
# class DateRange(TypedDict):
#     start_date: str
#     end_date: str


llm = init_chat_model(model="gpt-4o-mini", temperature=0)

structured_llm = llm.with_structured_output(DateRange)

prompt = ChatPromptTemplate.from_messages([
    ("system", DATE_EXTRACTOR_PROMPT_TEMPLATE),
    ("human", "{user_query}")
])


user_query = "I need time off for the next 4 days"
todays_date = datetime.now().strftime("%Y-%m-%d")


chain = prompt | structured_llm

llm_response = chain.invoke({'todays_date': todays_date, 'user_query': user_query})

llm_response


DateRange(start_date='2026-01-08', end_date='2026-01-11')

## 6) Tool Calling

LLMs are great at writing poetry, but they are often bad at math and they don't know the current weather.
Tool calling is like giving the AI **specialized gadgets** to help it do its job.
You tell the AI: *"I am giving you a calculator tool. If you see a math problem, don't guess—use the tool."*
The AI decides **when** it needs help and **which** gadget to pick to solve the problem.

###  Code Context
In the code below:
* `@tool`: This is a decorator. It puts a "sticker" on your Python function that says *"Hey AI, you are allowed to use this!"*
* `llm.bind_tools(tools)`: This effectively hands the manual of available tools to the "Brain."
* **Important:** When the model uses a tool, it doesn't return a text answer immediately. It returns a `tool_call` (a request to run the code). You (the code) must then run that function and give the answer back to the AI.

📚 **[Docs: Tool Calling](https://docs.langchain.com/oss/python/langchain/models#tool-calling)**


In [21]:
from langchain.tools import tool
from langchain_core.messages import HumanMessage
import math

@tool
def circle_area(radius: float) -> float: 
    """Return the area of a circle for a given radius."""
    return math.pi * radius * radius

tools = [circle_area]

# Bind tools to the model so it can decide to call them
llm_with_tools = llm.bind_tools(tools)

msg = HumanMessage(content="I have a circular garden of radius 3. What's the area?")
ai_msg = llm_with_tools.invoke([msg])

print("Model output Type", type(ai_msg))
print("Model output", ai_msg)
print("Tool calls (if any):", getattr(ai_msg, "tool_calls", None))



Model output Type <class 'langchain_core.messages.ai.AIMessage'>
Model output content='' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 62, 'total_tokens': 76, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_c4585b5b9c', 'id': 'chatcmpl-CvXxWlSfceAZ32wnXRYNx81CnmKlM', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None} id='lc_run--019b9af4-6fde-70c2-a058-9a14ba0d53c4-0' tool_calls=[{'name': 'circle_area', 'args': {'radius': 3}, 'id': 'call_5a82gzLMw3qN7cZQw20u8ulb', 'type': 'tool_call'}] invalid_tool_calls=[] usage_metadata={'input_tokens': 62, 'output_tokens': 14, 'total_tokens': 76, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_toke

In [ ]:
# If a tool call is present, execute it and return a final answer:

final = None

if getattr(ai_msg, "tool_calls", None):
    for tool_call in ai_msg.tool_calls:
        if tool_call["name"] == "circle_area":
            r = float(tool_call["args"]["radius"])
            result = circle_area.invoke({"radius": r})
            final = f"The area is approximately {result:.2f} square units."
else:
    final = ai_msg.content

print("Final:", final)

Final: The area is approximately 28.27 square units.


## 6.1) Tool Calling - MCP (Model Context Protocol)

**Note:** MCP requires Python 3.10+. If you're using Python 3.9, skip this section - you've already learned the core concept of tool calling above!

Standard tool calling (above) works great for tools defined in your own code. But what if you want to use a tool living on a different server or a database?
**MCP** is like a **Universal USB Standard** for AI tools. It allows your LangChain app to plug into external servers to fetch tools, without you having to write the tool logic yourself.

###  Code Context
In the code below:
* `MultiServerMCPClient`: Connects to a local script (`mcp_server.py`) acting as a tool provider.
* `client.get_tools()`: Automatically fetches the available tools from that server so the LLM can use them.

📚 **[Docs: Model Context Protocol](https://docs.langchain.com/oss/python/langchain/mcp)**

In [ ]:
# MCP - Model Context Protocol
# This feature works with Python 3.10+

# Try real imports first, otherwise provide a lightweight local stub so the notebook can continue.
try:
    from langchain_mcp_adapters.client import MultiServerMCPClient
except Exception:
    try:
        # alternate import path some installs might expose
        from langchain.mcp.adapters.client import MultiServerMCPClient  # type: ignore
    except Exception:
        import warnings
        warnings.warn(
            "Could not import MultiServerMCPClient from langchain_mcp_adapters. "
            "Falling back to a local stub. Install 'langchain-mcp-adapters' to use real MCP features."
        )

        class MultiServerMCPClient:
            """
            Minimal stub replacement for MultiServerMCPClient used in the notebook.
            This stub supports the async get_tools() call and returns an empty list.
            Replace with the real implementation by installing `langchain-mcp-adapters`.
            """

            def __init__(self, servers: dict):
                self.servers = servers

            async def get_tools(self):
                # Return an empty tool list so subsequent code can run without MCP.
                return []


client = MultiServerMCPClient(  
    {
        "area": {
            "transport": "stdio",  # Local subprocess communication
            "command": "python",
            "args": ["./mcp_server.py"]  # Path to your mcp_server.py file
        }
    }
)

tools = await client.get_tools()  

tools # You can now bind tools to the model as we did before

/var/folders/w9/06q0g35n6nv7cdc67p24ptr80000gn/T/ipykernel_53605/4190658928.py:13: UserWarning: Could not import MultiServerMCPClient from langchain_mcp_adapters. Falling back to a local stub. Install 'langchain-mcp-adapters' to use real MCP features.
  warnings.warn(


[]

## 7) RAG (Retrieval Augmented Generation)

Standard LLMs take tests from memory (and might "hallucinate" if they haven't studied). **RAG** allows the AI to take an **"Open Book" Test**.

1. **Loader:** We buy the textbook (e.g., download the SpaceX Wikipedia page).
2. **Splitter:** We rip the pages into small paragraphs so they are easier to handle.
3. **Vector Store:** We file these paragraphs in a cabinet organized by *meaning* (numerical vectors).
4. **Retriever:** When you ask a question, we find the specific paragraphs relevant to that question and give them to the AI to read before answering.


![Diagram](./images/Basic_Rag.png)

###  Code Context
In the code below:
* `WebBaseLoader`: A tool to scrape the text from the SpaceX Wikipedia page.
* `RecursiveCharacterTextSplitter`: Cuts the long text into chunks of 1000 characters so they fit in the model's context window.
* `InMemoryVectorStore`: A temporary database that stores the "meaning" of the text chunks.
* `rag_chain`: The final pipeline that takes your question → finds docs → sends both to the LLM → gives you the answer.

📚 **Official Documentation:**
* [Document Loaders](https://python.langchain.com/docs/integrations/document_loaders/)
* [Text Splitters](https://python.langchain.com/docs/concepts/#text-splitters)
* [Retrievers](https://python.langchain.com/docs/concepts/#retrievers)
* [Vector Stores](https://python.langchain.com/docs/integrations/vectorstores/)


In [29]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_prompts import RAG_PROMPT_TEMPLATE

ModuleNotFoundError: No module named 'langchain_community'

In [28]:
  # 1. Load the SpaceX Wikipedia page using a LangChain document loader
print("Loading Wikipedia page...")
loader = WebBaseLoader(
    web_paths=("https://en.wikipedia.org/wiki/SpaceX",),
)
docs = loader.load()
docs

Loading Wikipedia page...


NameError: name 'WebBaseLoader' is not defined

In [17]:
# 2. Split docs into smaller chunks
splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
)

chunks = splitter.split_documents(docs)

In [18]:
# 3. Create an in-memory vector store
embeddings = OpenAIEmbeddings()
vector_store = InMemoryVectorStore(embeddings)

# Add the chunks to the in-memory store
vector_store.add_documents(chunks)

['c1dfc55a-f1b5-44f7-9e36-96372ccfbcc4',
 'bc258d74-bae7-40a8-a2cd-901724aa6bd7',
 'ad9a23a7-be9a-45b9-acad-7728b2565a4a',
 'c6fdd12c-b338-47a0-9529-6c7c3cc32641',
 'a40d85a5-dbd0-4ce1-9e60-7c2cf61970a2',
 '5efd361b-154e-4d34-b2e0-590daf6f2d64',
 '6f906fa9-51f2-4132-a6c2-3b4eff3bf07c',
 '13d19bcf-9c67-4b8e-8f65-c1a8bfe39e8e',
 '94da3798-7365-49a9-b592-e1301264d8d5',
 '318d2dc0-ee1b-44c7-b443-1559968e3a85',
 'ad9777a6-b404-41a6-abf3-4a4ce0e1902b',
 '8a3fd781-3c87-4d32-aeb9-69c3649f4cda',
 'bee10e44-09a4-4618-a48a-addac35e4a12',
 '7ff2187d-cf7e-4e30-935e-e849adf88342',
 '2266f744-f267-4bfe-89ee-b69b386a75fe',
 '847c5c63-4391-4e0d-a540-8be0ff0ec0bf',
 '774304c7-a5ab-4a0e-9d93-9dedad040c7c',
 '70dc3b65-8df5-4b3c-9bed-3ebdf648c41b',
 '2dbae181-eb5f-4c98-990d-6aeaec8ac168',
 'f1c6054d-4712-451f-a97e-a263a85c060d',
 '7a06e11e-f7f2-4a72-a1b5-a323cb8462b8',
 '4f7c381e-d25c-466c-a1e5-48ec06aa313a',
 '9421a2ab-c1be-4a24-9112-3d09fc8927ce',
 '8579e684-2cba-452f-b749-acbf1b22453d',
 'a92dfb65-1c39-

In [20]:
# 4. Define a retreiven for the vector store (top-k)
retriever = vector_store.as_retriever(
    search_kwargs={"k": 5}
    )

# Helper to join document contents into a single string
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [21]:
user_query = "when was SapceX founded"
retriever.invoke(user_query)

[Document(id='1b400f5a-1214-4b82-aefb-6d8a6b51f4b4', metadata={'source': 'https://en.wikipedia.org/wiki/SpaceX', 'title': 'SpaceX - Wikipedia', 'language': 'en'}, page_content="SpaceX's 2024 Polaris Dawn mission featured the first-ever private spacewalk, marking a major milestone in commercial space exploration.[102]\nIn 2025, ProPublica reported that Chinese investors were investing in SpaceX via offshore entities, such as the Cayman Islands. Experts speculated that this might raise national security concerns with regulators.[103] Later in 2025 ProPublica reported that Chinese equity ownership in SpaceX likely extended to direct investment, citing the court testimony of investor Iqbaljit Kahlon.[104]\nBy July 2025, as part of $5 billion equity raise SpaceX agreed to invest $2 billion in xAI.[105]"),
 Document(id='5efd361b-154e-4d34-b2e0-590daf6f2d64', metadata={'source': 'https://en.wikipedia.org/wiki/SpaceX', 'title': 'SpaceX - Wikipedia', 'language': 'en'}, page_content='Space Explo

In [22]:
# 5. RAG Q/A Chain

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
prompt = ChatPromptTemplate.from_template(RAG_PROMPT_TEMPLATE)

rag_chain = (
        {
            "context": retriever | RunnableLambda(format_docs),
            "question": RunnablePassthrough(),
        }
        | prompt
        | llm
    )


In [23]:
user_query = "when was SapceX founded"

In [24]:
result = rag_chain.invoke(user_query)
result.content


'SpaceX was founded on March 14, 2002, in El Segundo, California, U.S. The company was established by Elon Musk. Since its inception, SpaceX has made significant advancements in various areas of aerospace technology.'

### 📚 Recommended Resources
* **[LangChain Python Tutorials](https://python.langchain.com/docs/tutorials/)**: The official step-by-step guides.
* **[LangSmith](https://smith.langchain.com/)**: A tool to trace and debug your chains (vital for production).
* **[LangGraph Documentation](https://langchain-ai.github.io/langgraph/)**: The future of building complex agents.
* **[LangChain YouTube Channel](https://www.youtube.com/@LangChain)**: Great for visual learners and deep dives into specific topics.
* **[Usecases](https://docs.langchain.com/oss/python/learn)**: Explore real-world usecases